## Desafio - Regressão Linear Múltipla - Cursos Desenvolvimento de IA - RocketSeat

### Sobre o desafio

O objetivo deste exercício é construir um modelo de machine learning capaz de prever o valor do aluguel de um imóvel com base em suas características. Através da análise de um dataset de imóveis, você irá:

1. **Explorar os dados:** Realizar uma análise exploratória dos dados para entender a distribuição das variáveis, identificar possíveis outliers e correlações.
2. **Preparar os dados:** Limpar os dados, tratar valores ausentes e codificar variáveis categóricas.
3. **Construir um modelo:** Utilizar um algoritmo de regressão linear para construir um modelo que relacione as características do imóvel com o valor do aluguel.
4. **Avaliar o modelo:** Avaliar a performance do modelo utilizando métricas adequadas e analisar os resíduos para verificar a qualidade das previsões.
5. **Interpretar os resultados:** Analisar os coeficientes do modelo para entender a importância de cada variável na previsão do valor do aluguel.

In [ ]:
# Importar bibliotecas gerais
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import numpy as np
from scipy.stats import zscore
import pingouin as pg

# Importando módulos do scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Imports necessários para testes de normalidade
from scipy.stats import shapiro, kstest, anderson
from statsmodels.stats.diagnostic import lilliefors, het_goldfeldquandt

### Etapas:

1. **Carregar e explorar o dataset:** 
Carregue o arquivo CSV fornecido e explore os dados utilizando as funções do pandas.

In [ ]:
# Criando dataframe
df_aluguel = pd.read_csv('./datasets/dataset_aluguel.csv')


In [ ]:
# Checagem do dataset
df_aluguel.info()

In [ ]:
# Remover coluna Id
df_aluguel.drop(columns='id', axis=1, inplace=True)

In [ ]:
# Checagem do dataset após remover a coluna id
df_aluguel.info()

In [ ]:
# Primeiras linhas do DataFrame
df_aluguel.head()

In [ ]:
# Últimas linhas do DataFrame
df_aluguel.tail()

In [ ]:
# Dimensões do DataFrame
df_aluguel.shape

In [ ]:
# Estatísticas descritivas básicas
df_aluguel.describe()

In [ ]:
# Matriz de correlação usando mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(df_aluguel.corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
# Histograma geral das variáveis numéricas
df_aluguel.hist(figsize=(10, 8))

2. **Pré-processamento:** Realize as seguintes tarefas:
    - Identifique e trate valores ausentes.
    - Remova outliers.
    - Codifique as variáveis categóricas.
    - Escale as variáveis numéricas (se necessário).

In [ ]:
# Verificar valores ausentes
df_aluguel.isnull().sum()

In [ ]:
# Codificar variáveis categóricas
df_aluguel = pd.get_dummies(df_aluguel, columns=['localizacao_Periferia', 'localizacao_Subúrbio'], dtype='int64')

In [ ]:
# Box plot para detectar outliers
sns.boxplot(data=df_aluguel, x='tamanho_m2')
plt.show()
sns.boxplot(data=df_aluguel, x='n_quartos')
plt.show()
sns.boxplot(data=df_aluguel, x='idade_casa')
plt.show()
sns.boxplot(data=df_aluguel, x='garagem')
plt.show()
sns.boxplot(data=df_aluguel, x='valor_aluguel')
plt.show()

In [ ]:
# Visualizar relações entre pares de variáveis numéricas
sns.pairplot(df_aluguel)

3. **Construção do modelo:**
    - Divida os dados em conjuntos de treino e teste.
    - Crie um pipeline para pré-processar os dados e treinar o modelo de regressão linear.
    - Treine o modelo utilizando o conjunto de treino.

In [ ]:
# Criando conjunto de treino e teste
X = df_aluguel.drop(columns='valor_aluguel', axis=1)
y = df_aluguel['valor_aluguel']

In [ ]:
# Dividindo conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [ ]:
# Definir colunas numéricas
colunas_numericas = ['tamanho_m2', 'n_quartos', 'idade_casa', 'garagem']

In [ ]:
# Criar um Transformer de variáveis numéricas
transformer_numericas = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [ ]:
# Criar um ColumnTransformer que encapsula todas as transformações
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numericas, colunas_numericas)
    ]
)

In [ ]:
# Pipeline principal = Pre Processamento + Treinamento
model_regr = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', LinearRegression())])

In [ ]:
# Treinar modelo
model_regr.fit(X_train, y_train)

4. **Avaliação do modelo:**
    - Faça previsões sobre o conjunto de teste.
    - Calcule métricas de desempenho (R², MAE, RMSE).
    - Analise os resíduos para verificar a qualidade do modelo.

In [ ]:
# Gerar predição
y_pred = model_regr.predict(X_test)
y_pred

In [ ]:
# Criar DataFrame com valores reais e previstos
resultados = pd.DataFrame({
    'Valor Real': y_test,
    'Valor Previsto': y_pred,
    'Diferença': y_test - y_pred
})

In [ ]:
# Mostrar primeiras linhas dos resultados
print("Comparação entre valores reais e previstos:")
print(resultados.head())

In [ ]:
# Estatísticas básicas das previsões
print("\nEstatísticas das previsões:")
print(resultados.describe())

In [ ]:
# Visualizar distribuição das diferenças
plt.figure(figsize=(10, 6))
plt.hist(resultados['Diferença'], bins=30)
plt.title('Distribuição dos Erros de Previsão')
plt.xlabel('Diferença (Real - Previsto)')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Calcular métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: R$ {mae:.2f}")
print(f"RMSE: R$ {rmse:.2f}")
print(f"R²: {r2:.3f}")

In [ ]:
# Calcular resíduos (erros do modelo)
residuos = y_test - y_pred
residuos

In [ ]:
# Transformar os resíduos na escala padrão -> (X - média) / desvio_padrao
residuos_std = zscore(residuos)
residuos_std

In [ ]:
# Verificar linearidade dos resíduos: valores entre -2 e +2 (escala padrão)
# Verificar homocedasticidade: valores em torno da reta
sns.scatterplot(x=y_pred, y=residuos_std)
plt.axhline(y=0)
plt.axhline(y=-2)
plt.axhline(y=2)

In [ ]:
# Checar se resíduos seguem uma distribuição normal
# qq plot 
plt.figure(figsize=(15, 8))
pg.qqplot(residuos_std, dist='norm', confidence=0.95)
plt.xlabel('Quantis teóricos')
plt.ylabel('Resíduos na escala padrão')
plt.show()

5. **Interpretação dos resultados:**
    - Analise os coeficientes do modelo para entender a importância de cada variável.
    - Discuta as limitações do modelo e sugira possíveis melhorias.

In [ ]:
# Teste de normalidade de Shapiro-Wilk 
stat_shapiro, p_value_shapiro = shapiro(residuos)
print("Estatistica do Teste: {} e P-Value: {}".format(stat_shapiro, p_value_shapiro))

In [ ]:
# Teste de normalidade de Kolmogorov-Smirnov
stat_ks, p_value_ks = kstest(residuos, 'norm')
print("Estatistica do Teste: {} e P-Value: {}".format(stat_ks, p_value_ks))

In [ ]:
# Teste de normalidade de Liliefors
stat_ll, p_value_ll = lilliefors(residuos, dist='norm', pvalmethod='table')
print("Estatistica do Teste: {} e P-Value: {}".format(stat_ll, p_value_ll))

In [ ]:
# Teste de normalidade de Anderson-Darling
stat_and, critical_and, significance_and = anderson(residuos, dist='norm')

In [ ]:
# Visualizar valores do valor critico para o teste de normalidade de Anderson-Darling
critical_and

In [ ]:
# Visualizar valores do valor de significância para o teste de normalidade de Anderson-Darling
significance_and

In [ ]:
# Validando a estatistica do teste com o valor critico
print("Estatistica do Teste: {} e Valor Crítico: {}".format(stat_and, critical_and[2]))

In [ ]:
# Teste de Homocedasticidade Goldfeld-Quandt
pipe = Pipeline(steps=[('preprocessor', preprocessor)])
X_test_transformed = pipe.fit_transform(X_test)

In [ ]:
# Consultar Pipeline criada
X_test_transformed

In [ ]:
# Executar teste Goldfeld-Quandt
test_goldfeld = het_goldfeldquandt(residuos, X_test_transformed)
stat_goldfeld = test_goldfeld[0]
p_value_goldfeld = test_goldfeld[1]
print("Estatistica de goldfeld: {} e P-Value: {}".format(stat_goldfeld, p_value_goldfeld))

In [ ]:
predicao_individual = {
    'tamanho_m2': 106.18101782710437,
    'n_quartos': 1,
    'idade_casa': 36.7608059620386,
    'garagem': 0,
    'localizacao_Periferia': True,
    'localizacao_Subúrbio': False
}

sample_df = pd.DataFrame(predicao_individual, index=[1])

In [ ]:
# Visual Dataframe
sample_df

In [ ]:
# Predição
model_regr.predict(sample_df)

In [ ]:
# Salvar modelo para criar um UI de predição com o Gradio
joblib.dump(model_regr, './modelo_aluguel.pkl')

### Discussões

Para um modelo mais preciso e com uma melhor capacidade de predição, seria interessante adicionar no dataset os seguintes itens:
- número de banheiros;
- área útil;
- presença de algo relacionado à lazer (pscina, churrasqueira, quadra, etc);
- com mobilia ou sem;
- distância até o centro;
- o que há nas proximidades;
- bairro (está muito amplo apenas subúrbio e periferia);
- índice de segurança da região;
- duração do contrato;
- tipo de garantia exigida (fiador, caução, etc);
- valor de IPTU;

In [ ]:
# Novo formato do dataset com colunas adicionais
novo_dataset = {
    'id': [],
    'tamanho_m2': [],
    'area_util': [],
    'n_quartos': [],
    'n_banheiros': [],
    'idade_casa': [],
    'garagem': [],
    'dist_metro': [],   # em km
    'dist_centro': [],  # em km
    'bairro': [],      # nome do bairro
    'indice_seguranca': [], # escala 0-100
    'itens_lazer': [],  # lista de amenidades
    'mobiliado': [],   # 0:não, 1:sim
    'valor_iptu': [],
    'duracao_contrato': [], # em meses
    'tipo_garantia': [], # 1:fiador, 2:seguro, 3:caução
    'valor_aluguel': []
}

Estas melhorias tornariam o modelo mais robusto por:

Capturar mais dimensões que influenciam o preço
Permitir análises mais granulares
Melhorar a capacidade de generalização
Reduzir o erro de previsão
Aumentar a interpretabilidade do modelo
O dataset atual, embora funcional, deixa de capturar muitas nuances importantes do mercado imobiliário que poderiam melhorar significativamente a precisão das previsões.